可以用的程式碼在第一格，其他為測試用

In [1]:
import tensorflow as tf

graph = tf.get_default_graph()
with graph.as_default():
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph('./save_model/final-model.ckpt.meta')
        saver.restore(sess, './save_model/final-model.ckpt')
        input_tensor = graph.get_tensor_by_name('input:0')
        output_tensor = graph.get_tensor_by_name('output:0')   
        converter = tf.lite.TFLiteConverter.from_session(sess, [input_tensor], [output_tensor])
        tflite_model = converter.convert()
    with open('./final-model.tflite', 'wb') as f:
        f.write(tflite_model)


2023-08-23 16:50:54.625747: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 16:50:55.959274: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-23 16:50:55.959347: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-23 16:51:05.158062: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

底下為測試用

In [7]:
import tensorflow as tf
graph = tf.Graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('./save_model/final-model.ckpt.meta')
    saver.restore(sess, './save_model/final-model.ckpt')

    input_tensor = graph.get_tensor_by_name('input:0')  
    #取出input張量。'input'對應到儲存模型時的name參數。':0'代表第幾個，一定要加
    output_tensor = graph.get_tensor_by_name('output:0')
    #output同理

    converter = tf.lite.TFLiteConverter.from_session(sess, input_tensors=[input_tensor], output_tensors=[output_tensor])
    tflite_model = converter.convert()
    with open('converted_model.tflite', 'wb') as f:
        f.write(tflite_model)

INFO:tensorflow:Restoring parameters from ./save_model/final-model.ckpt


KeyError: "The name 'input:0' refers to a Tensor which does not exist. The operation, 'input', does not exist in the graph."

In [ ]:
import tensorflow as tf
import numpy as np
inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
targets = np.array([[0], [0], [0], [1]], dtype=np.float32)

# 建立模型
x = tf.placeholder(tf.float32, shape=[None, 2], name='input')
y_true = tf.placeholder(tf.float32, shape=[None, 1], name='output')

weights = tf.Variable(tf.random_normal([2, 1]))
bias = tf.Variable(tf.random_normal([1]))

logits = tf.add(tf.matmul(x, weights), bias)
y_pred = tf.sigmoid(logits)

# 計算損失和訓練操作
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y_true))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss)
# 建立 TensorFlow 會話
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 訓練模型
    for epoch in range(1000):
        _, current_loss = sess.run([train_op, loss], feed_dict={x: inputs, y_true: targets})
        if epoch % 100 == 0:
            print("Epoch {}: Loss = {}".format(epoch, current_loss))
            
    save_path = saver.save(sess, './save_model/final-model.ckpt')
    # 使用模型進行預測
    predictions = sess.run(y_pred, feed_dict={x: inputs})
    print("Predictions:", predictions)
model_path = "./tf1_model.ckpt"
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, model_path)

# 轉換 TensorFlow 1.x 模型至 TensorFlow Lite 格式
converter = tf.lite.TFLiteConverter.from_session(sess, [x], [y_pred])
tflite_model = converter.convert()

# 儲存 TensorFlow Lite 模型
tflite_model_path = "./model.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)


Epoch 0: Loss = 0.6322017908096313
Epoch 100: Loss = 0.4315967857837677
Epoch 200: Loss = 0.320986807346344
Epoch 300: Loss = 0.252130925655365
Epoch 400: Loss = 0.20472589135169983
Epoch 500: Loss = 0.1698969304561615
Epoch 600: Loss = 0.14323517680168152
Epoch 700: Loss = 0.12225791066884995
Epoch 800: Loss = 0.10542244464159012
Epoch 900: Loss = 0.09170080721378326
Predictions: [[0.00139599]
 [0.10193919]
 [0.08695634]
 [0.88549376]]
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph


RuntimeError: Attempted to use a closed Session.

In [ ]:
import tensorflow as tf
import numpy as np
with tf.Graph().as_default():
    with tf.Session() as sess:
        # 讀取模型
        saver = tf.train.import_meta_graph('./save_model/final-model.ckpt.meta')
        saver.restore(sess, './save_model/final-model.ckpt')

        # 提取輸入和輸出張量
        input_tensor = tf.get_default_graph().get_tensor_by_name('input:0')
        output_tensor = tf.get_default_graph().get_tensor_by_name('output:0')  # 根據模型結構更改

        # 轉換 TensorFlow 1.x 模型至 TensorFlow Lite 格式
        converter = tf.lite.TFLiteConverter.from_session(sess, [input_tensor], [output_tensor])
        converter.post_training_quantize = True
        converter.allow_custom_ops = True
        tflite_model = converter.convert()

    # 儲存 TensorFlow Lite 模型
    with open('./final-model.tflite', 'wb') as f:
        f.write(tflite_model)

INFO:tensorflow:Restoring parameters from ./save_model/final-model.ckpt

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.


2023-08-22 09:38:46.666005: I tensorflow/core/grappler/devices.cc:60] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA support)
2023-08-22 09:38:46.666817: I tensorflow/core/grappler/clusters/single_machine.cc:359] Starting new session
2023-08-22 09:38:46.690167: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:716] Optimization results for grappler item: graph_to_optimize
2023-08-22 09:38:46.690340: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:718]   function_optimizer: function_optimizer did nothing. time = 0.005ms.
2023-08-22 09:38:46.690374: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:718]   function_optimizer: function_optimizer did nothing. time = 0.001ms.
/home/nrnmnrn/tf1_env/lib/python3.7/site-packages/tensorflow/lite/python/lite.py:769: UserWarning: Property post_training_quantize is deprecated, please use optimizations=[Optimize.DEFAULT] instead.
  " instead." % name)
2023-08-22

ConverterError: TOCO failed. See console for info.
/home/nrnmnrn/tf1_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nrnmnrn/tf1_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nrnmnrn/tf1_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nrnmnrn/tf1_env/lib/python3.

In [9]:
!pip install numpy==1.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 1.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [13]:
tf.compat.v1.reset_default_graph()  # 重置計算圖，準備轉換
with tf.Graph().as_default():
    with tf.Session() as sess:
        # 載入模型
        saver = tf.train.import_meta_graph('./save_model/final-model.ckpt.meta')
        saver.restore(sess, './save_model/final-model.ckpt')

        # 獲取輸入和輸出 tensors
        input_tensor = tf.get_default_graph().get_tensor_by_name("input:0")  # 請替換成您模型的輸入名稱
        output_tensor = tf.get_default_graph().get_tensor_by_name("output:0")  # 請替換成您模型的輸出名稱

        # 轉換 TensorFlow 1.x 模型至 TensorFlow Lite 格式
        converter = tf.lite.TFLiteConverter.from_session(sess, [input_tensor], [output_tensor])
        tflite_model = converter.convert()

# 儲存 TensorFlow Lite 模型
tflite_model_path = "./model.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Restoring parameters from ./save_model/final-model.ckpt


TypeError: get_tensor_by_name() got an unexpected keyword argument 'dtype'